ログ用画像保存先指定

In [1]:
import os
import datetime

blocked_dir = 'local_test/blocked'
free_dir = 'local_test/free'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')
    
try:
    with open('local_test/' + datetime.datetime.now().strftime('%Y%m%d')+'_log.txt','x') as f:
        f.write('image_name' +  '	' + 'classification' + '	' +'prob_blocked')
except FileExistsError:
    print('Logfile already exist')    

#Load the trained model(学習済みモデルの読み込み)

We'll assumed that you've already downloaded the best_model.pth to your workstation as instructed in the training notebook. Now, you should upload this model into this notebook's directory by using the Jupyter Lab upload tool. Once that's finished there should be a file named best_model.pth in this notebook's directory. 

(追加) ローカルの学習なので、すでにbest_model.pthがローカルに存在している事を前提とします。


Please make sure the file has uploaded fully before calling the next cell

Execute the code below to initialize the PyTorch model. This should look very familiar from the training notebook.

(訳) Pytorch modelの初期化を下記コードでおこないます。


In [2]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

次に、best_model.pthから、学習した重みを読み込みます

In [3]:
model.load_state_dict(torch.load('../model/best_model.pth'))

現在、モデルの重みは、CPUメモリーで実行されているので、下記コードでGPUに転送します。

In [4]:
device=torch.device('cuda')
model=model.to(device)

Create the preprocessing function(事前処理関数の作成)

We have now loaded our model, but there's a slight issue. The format that we trained our model doesnt exactly match the format of the camera. To do that, we need to do some preprocessing. This involves the following steps

(訳) モデルを読み込みましたが、まだわずかな問題があります。学習済みモデルのフォーマットと、カメラの形式と完全に一致しません。これを解消するために、 いくつかの前処理を行う必要があります。これらは、下記の手順になります。
1.Convert from BGR to RGB
2.Convert from HWC layout to CHW layout
3.Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
4.Transfer the data from CPU memory to GPU memory
5.Add a batch dimension

(訳)
1.BGRからRGBに変換
2.HWC layoutからCHW layoutに変換
3.トレーニング中に使ったのと同じパラメーターを使用して正規化します（カメラは[0、255]の範囲の値を提供し、ロードされた画像は[0、1]の範囲でトレーニングするため、255.0でスケーリングする必要があります
4.dataをCPUメモリからGPUメモリに転送します
5.バッチディメンションを追加する

In [5]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

画像保存メソッドの定義

In [6]:
import datetime

def save_image(directory,im_name):
    cap_image=image
    image_path = os.path.join(directory, im_name)
    with open(image_path, 'wb') as f:
        f.write(cap_image.value)

def save_log(classification):
    global im_name,blocked_slider
    write_log=open(os.path.join('log', datetime.datetime.now().strftime('%Y%m%d')+'_log.txt'),'a')
    write_log.write('\n'+im_name +  '	' + classification+  '	' + str(blocked_slider.value))
    write_log.close()
    
def capture_snapshot(blocked_val):
    global free_dir,blocked_dir,blocked_slider
    if blocked_val < 0.5:
        save_snapshot(free_dir)
        save_log('free')
    else:
        save_snapshot(blocked_dir)
        save_log('blocked')        

各ボタン動作定義

In [7]:
import torch.nn.functional as F

def determine_image(test_image):
    global blocked_slider
    x = test_image 
    x = preprocess(x)
    y = model(x)
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    prob_blocked = float(y.flatten()[0])
    return prob_blocked

フォルダセレクト,テスト実行

In [51]:
import glob
import cv2

count_error=0
files =glob.glob("../dataset/free/*")

for fname in files: 
    test_img=cv2.imread(fname)
    val_prob_blocked=determine_image(test_img)
    if val_prob_blocked>0.5:
        print(fname)
        print(val_prob_blocked)
        count_error=count_error+1

print(count_error/len(files)*100)

../dataset/free/8f7fca6e-a16b-11ea-ae2f-72b5f773b75d.jpg
0.6890029907226562
../dataset/free/1cb7854e-a16b-11ea-ae2f-72b5f773b75d.jpg
0.9693331122398376
1.1111111111111112
